<a href="https://colab.research.google.com/github/jvplopes/Gr-fico-de-mapa/blob/main/China_world_trade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Parceiros comerciais da China | 2019

Neste notebook, os dados comerciais da China no ano de 2019 serão utilizados como exemplo para criar um gráfico de mapa.

Instalação do API e importação das bibliotecas necessárias

In [ ]:
#Bibliotecas necessárias para o dataframe
#world_trade_data API
!pip install world_trade_data --upgrade

#Importação do API
import world_trade_data as wits

In [ ]:
#Bibliotecas necessárias para o gráfico de mapa
import plotly.graph_objects as go
import pandas as pd

Lista com todos os países disponíveis.

In [ ]:
#Todos os países fornecidos pelo API
countries = wits.get_countries()

A função `countries_data_load` alimenta a lista `countries_data` com os dados que serão usados (nome do país parceiro, iso code 3, total de importação, total de exportação)

In [ ]:
#Lista com os dados dos países (nome, iso code, total de importação, total de exportação)
countries_data = []

In [ ]:
#Carrega a lista coutries_data com os dados de importação e exportação
def countries_data_load(countries):
  for index, row in countries.iterrows():
    #Nome e Iso code 3
    name = row[0] #Nome do país
    code = row.name #Iso code 3 para fazer o gráfico de mapa

    #Obtenção das importações por país
    try:
      imports = wits.get_indicator('MPRT-TRD-VL', reporter='chn', year='2019', partner=code)

      #valores das importaçãos
      valores = imports.Value

      #Valor máximo, que é o All products, multiplicado por mil em razão do valor estar reduzido em mil no banco de dados.
      imports_total = valores.max() *1000

    except:
      #Os países que não estiveres listados entre os parceiros será salvo com o valor 0.
      imports_total = 0
    
    
    #Obtenção das exportações por país
    try:
      exports = wits.get_indicator('XPRT-TRD-VL', reporter='chn', year='2019', partner=code)

      #valores das exportações
      valores = exports.Value 

      #Valor máximo, que é o All products
      exports_total = valores.max() *1000
    except:
      
      exports_total = 0

    countries_data.append([code, name, imports_total, exports_total]) #insere os dados na lista de dados dos países

countries_data_load(countries)

In [ ]:
#Cria o dataframe com os valores dos países
df_partners = pd.DataFrame(data=countries_data, index=None, columns=['iso_code_3', 'partner', 'import_total', 'export_total'])

In [ ]:
# Remoção de entradas que não são países
excluidos = ['World', 'Europe & Central Asia', 'East Asia & Pacific', 
             'North America	', 'Latin America & Caribbean	', 'Middle East & North Africa',
             'North America', 'South Asia']

df_partners = df_partners.drop(df_partners.loc[df_partners['partner'].isin(excluidos)].index)

#Remove alguns valores duplicados
df_partners.drop_duplicates(subset=['iso_code_3'])

,iso_code_3,partner,import_total,export_total
0,AFG,Afghanistan,2.928787e+07,6.009975e+08
1,ALB,Albania,1.037826e+08,5.985241e+08
2,DZA,Algeria,1.141660e+09,6.945571e+09
3,ASM,American Samoa,0.000000e+00,0.000000e+00
4,AND,Andorra,1.996050e+05,2.369482e+06
...,...,...,...,...
261,YEM,Yemen,8.644422e+08,2.821134e+09
262,YDR,Yemen Democratic,0.000000e+00,0.000000e+00
263,YUG,"Yugoslavia,FR(Serbia/Montenegr",0.000000e+00,0.000000e+00
264,ZMB,Zambia,3.298462e+09,9.703027e+08


In [ ]:
#Países com maiores valores de importação
imports_partners = df_partners.sort_values(by = "import_total", ascending = False)[0:5]

In [ ]:
#Países com maiores valores de exportação
exports_partners = df_partners.sort_values(by = "export_total", ascending = False)[0:5]

In [ ]:
#grafico mundial dos parceiros de importação a Rússia

fig = go.Figure(data=go.Choropleth(
    locations = df_partners['iso_code_3'],
    z = df_partners['import_total'],
    text = df_partners['partner'],
    colorscale = 'blues',
    autocolorscale=False,
    reversescale=False,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_tickprefix = ' ',
    colorbar_title = 'US$'
))

fig.update_layout(
    title_text='Parceiros de importação da China | 2019',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='orthographic'))


fig.show()

In [ ]:
#grafico mundial dos parceiros de exportação a China

fig = go.Figure(data=go.Choropleth(
    locations = df_partners['iso_code_3'],
    z = df_partners['export_total'],
    text = df_partners['partner'],
    colorscale = 'reds',
    autocolorscale=False,
    reversescale=False,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_tickprefix = ' ',
    colorbar_title = 'US$'
))

fig.update_layout(
    title_text='Parceiros de exportação da China | 2019',
    geo=dict(
        showframe=False,
        showcoastlines=False,
        projection_type='natural earth'))


fig.show()